In [1]:
# importing Relevant Packages
import numpy as np 
import pandas as pd 
import gc
import pickle

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/jpx-my-baseline-skgb/__results__.html
/kaggle/input/jpx-my-baseline-skgb/simple_skgb.sav
/kaggle/input/jpx-my-baseline-skgb/submission.csv
/kaggle/input/jpx-my-baseline-skgb/__notebook__.ipynb
/kaggle/input/jpx-my-baseline-skgb/__output__.json
/kaggle/input/jpx-my-baseline-skgb/custom.css
/kaggle/input/jpx-my-baseline-transformer/__results__.html
/kaggle/input/jpx-my-baseline-transformer/submission.csv
/kaggle/input/jpx-my-baseline-transformer/__notebook__.ipynb
/kaggle/input/jpx-my-baseline-transformer/__output__.json
/kaggle/input/jpx-my-baseline-transformer/custom.css
/kaggle/input/jpx-my-baseline-transformer/tr_1.tf/saved_model.pb
/kaggle/input/jpx-my-baseline-transformer/tr_1.tf/keras_metadata.pb
/kaggle/input/jpx-my-baseline-transformer/tr_1.tf/variables/variables.index
/kaggle/input/jpx-my-baseline-transformer/tr_1.tf/variables/variables.data-00000-of-00001
/kaggle/input/jpx-my-baseline-dnn/__results__.html
/kaggle/input/jpx-my-baseline-dnn/submission.csv
/kaggle/i

## Loading the Gradient Boosting Regression Models 

In [2]:
#loading models
model_1 = pickle.load(open('../input/jpx-my-baseline-xgboost/simple_xgb.sav', 'rb'))
model_2 = pickle.load(open('../input/jpx-my-baseline-lgboost/simple_LGBM.sav', 'rb'))
model_3 = pickle.load(open('../input/jpx-my-baseline-skgb/simple_skgb.sav', 'rb'))

#setting the wieghts:
w = [0.25,0.7,0.05] 


[15:26:08] WARNING: ../src/gbm/gbtree.cc:390: Loading from a raw memory buffer on CPU only machine.  Changing tree_method to hist.
[15:26:08] WARNING: ../src/learner.cc:248: No visible GPU is found, setting `gpu_id` to -1


In [3]:
#loading DNN models

import keras

dnn_1 = keras.models.load_model("../input/jpx-my-baseline-dnn/dnn_1.tf")

2022-06-29 15:26:20.448843: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-06-29 15:26:21.432507: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [4]:
#loading Transformer

import keras

tr_1 = keras.models.load_model("../input/jpx-my-baseline-transformer/tr_1.tf")

## Feature Engeneering

In [5]:
# define function to compute log returns
# def log_return(series, periods=1):
#     return np.log(series).diff(periods=periods)


# # def fill_the_gaps(df):
# #     new_df = pd.DataFrame(columns= df.columns)
# #     for i in range(len(df['SecuritiesCode'].unique())):
# #         new_df = new_df.append(df[df['SecuritiesCode'] == i].reindex(range(df[df['SecuritiesCode'] == i].index[0],df[df['SecuritiesCode'] == i].index[-1]+60,60),method='pad'))
# #     new_df.fillna(method = 'pad', inplace = True) 
# #     return new_df


# def rsiFunc(prices, n=14):
#     deltas = np.diff(prices)
#     seed = deltas[:n+1]
#     up = seed[seed>=0].sum()/n
#     down = -seed[seed<0].sum()/n
#     rs = up/down
#     rsi = np.zeros_like(prices)
#     rsi[:n] = 100. - 100./(1.+rs)

#     for i in range(n, len(prices)):
#         delta = deltas[i-1] # cause the diff is 1 shorter

#         if delta>0:
#             upval = delta
#             downval = 0.
#         else:
#             upval = 0.
#             downval = -delta

#         up = (up*(n-1) + upval)/n
#         down = (down*(n-1) + downval)/n

#         rs = up/down
#         rsi[i] = 100. - 100./(1.+rs)

#     return rsi



def get_features(df):

#     df['log_return_5'] = log_return(df['Close'],periods=5)
#     df['log_return'] = log_return(df['Close'],periods=1)

    upper_shadow = lambda asset: asset.High - np.maximum(asset.Close,asset.Open)
    lower_shadow = lambda asset: np.minimum(asset.Close,asset.Open)- asset.Low

    df['upper_shadow'] = upper_shadow(df)
    df['lower_shadow'] = lower_shadow(df)
    
#     df['EMA_21'] = df['Close'].ewm(span=21).mean()
    
#     df['EMA_55'] = df['Close'].ewm(span=55).mean()
    
#     df['EMA_315'] = df['Close'].ewm(span=315).mean()
    
#     df['EMA_825'] = df['Close'].ewm(span=825).mean()
    
#     window = 7
    
#     no_of_std = 2
    
#     df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
#     df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
#     window = 5
    
#     df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
    
#     df[f'EMA_{window}_std'] = df['Close'].rolling(window=window).std()
    
#     df[f'EMA_{window}_BB_high'] = df[f'EMA_{window}'] + no_of_std * df[f'EMA_{window}_std']
    
#     df[f'MA_{window}MA_BB_low'] = df[f'EMA_{window}'] - no_of_std * df[f'EMA_{window}_std']
    
#     df['MACD'] = df['EMA_7'] - df['EMA_5']
    
    
#     df['rsi_5'] = rsiFunc(df['Close'].values, 5)
    
#     df['rsi_7'] = rsiFunc(df['Close'].values, 7)
    
#     df['rsi_21'] = rsiFunc(df['Close'].values, 21)
    
    
    
    df['VWAP'] = (df['Close'] * df['Volume'])/ df['Volume']




    
    
    #df = pd.concat([df, pd.get_dummies(df['Asset_ID'], prefix= 'Asset_')], axis=1)
    
    df[[ 'upper_shadow', 'lower_shadow','VWAP']].astype('float16')
    
    return df

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
#sellectiong the features GB

scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow','VWAP']
features = ["SecuritiesCode",'Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow','VWAP']

## Submiting

In [8]:
#Building the Ranking Function

def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

In [9]:
import jpx_tokyo_market_prediction


env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices = get_features(prices)
    prices = prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
    prices = prices.fillna(0)
    prices[scaled_features] = scaler.fit_transform(prices[scaled_features])
    pred_1 = model_1.predict(prices[features])
    pred_2 = model_2.predict(prices[features])
    pred_3 = model_3.predict(prices[features])
    prices['pred'] = pred_1*w[0] + pred_2*w[1] + pred_3*w[2]
    prices = add_rank(prices)
    feature_map = prices.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [10]:
# #with DNN

# import jpx_tokyo_market_prediction


# env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
# iter_test = env.iter_test()    # an iterator which loops over the test files
# for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
#     prices = get_features(prices)
#     prices = prices.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
#     prices = prices.fillna(0)
#     prices[tr_scaled_features] = scaler.fit_transform(prices[dnn_features])
#     prices['pred'] = dnn_1.predict(dict(prices))
#     prices = add_rank(prices)
#     feature_map = prices.set_index('SecuritiesCode')['Rank'].to_dict()
#     sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(feature_map)
#     env.predict(sample_prediction)

In [11]:

# sample = {
#     "SecuritiesCode": 33,
#     "Open": 0.00135,
#     "High": 0.1535,
#     "Low": 0.10035,
#     "Close": 0.13075,
#     "Volume": 0.1435,
#     "log_return": 0.00135,
#     "log_return_5": 0.00135,
#     "upper_shadow": 0.4135,
#     "lower_shadow": 0.0135,
#     "VWAP": 0.1305,
#     "EMA_7": 0.7135,
#     "EMA_7_std": 0.135,
#     "EMA_5": 0.135,
#     "EMA_5_std": 0.15,
#     "MACD": 0.125,
#     "rsi_5": 0.135,
#     "rsi_7": 0.015,
#     "rsi_21": 0,
# }

# input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}
# predictions = dnn_1.predict(input_dict)

# print(
#     "This particular patient had a %.1f percent probability "
#     "of having a heart disease, as evaluated by our model." % (100 * predictions[0][0],)
# )